In [1]:
#libraries
import numpy as np
import pandas as pd
import nltk
import string
from string import whitespace
import collections
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from nltk.tokenize import TweetTokenizer 

In [2]:
df = pd.read_csv('C:/Users/eugen/Desktop/IU_Data_Science/NLP/Twitter/twcs/twcs.csv',  sep=',')

In [3]:
df.shape

(2811774, 7)

In [4]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
df.isna().sum()

tweet_id                         0
author_id                        0
inbound                          0
created_at                       0
text                             0
response_tweet_id          1040629
in_response_to_tweet_id     794335
dtype: int64

In [6]:
organization = "TMobileHelp"

data = df.loc[df['author_id'] == organization]

In [29]:
data.shape

(34317, 7)

In [8]:
def preprocess_text(df):
    df['text'] = df.apply(lambda row: row['text'].lower(), axis=1)
    df['text'] = df.apply(lambda row: row['text'].rstrip('.'), axis=1)
    df['text'] = df.apply(lambda row: re.sub("\B@\w+", "", row['text']), axis=1)
    df['text'] = df.apply(lambda row: re.sub("\*\w+", "", row['text']), axis=1)

In [9]:
preprocess_text(data)

<ipython-input-8-8232c14b4411>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df.apply(lambda row: row['text'].lower(), axis=1)
<ipython-input-8-8232c14b4411>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df.apply(lambda row: row['text'].rstrip('.'), axis=1)
<ipython-input-8-8232c14b4411>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
data['text']

620         hey, trevor! send us a dm, and we'll take a c...
627          thanks for getting that dm sent over - i've ...
630         yay for t-mobile tuesdays- lets's get you tak...
632         well, we'd be happy to take a look at that an...
634         hey we are here and we want to help! if you h...
                                 ...                        
2811607     spooky! but don't forget to use #tmoboletuesd...
2811609      we want the best speeds for you where you ne...
2811611     hey chris! send a dm our way and i'd be happy...
2811613     i've just responded to your dm and look forwa...
2811615     that's weird, let's check that out. dm us, we...
Name: text, Length: 34317, dtype: object

In [11]:
tweet = data['text']
tweet = pd.Series(tweet).tolist()
tweet = ''.join(tweet)
tweet[:1000]

" hey, trevor! send us a dm, and we'll take a closer look!   thanks for getting that dm sent over - i've got it and am replying now. https://t.co/93xwcskfiy  yay for t-mobile tuesdays- lets's get you taken care of! send a dm my way: https://t.co/jgtdflsvbg  well, we'd be happy to take a look at that and see what's going on, jenny. dm us, we're here to keep you connected.  hey we are here and we want to help! if you have any questions feel free to shoot us a dm!  can i have you dm me at: https://t.co/ryd2pduspi  we want to help you with this great #tmobiletuesday deal! shoot me a dm here https://t.co/jab1jkomry\xa0  you are welcome and thanks for letting us know. if you need anything else we are here for you. #happytmobiletuesday  we want this to work perfectly! do you have at least 10 photos selected? if not, add another and try. dm w/ any questions!  :o creepy! happy halloween!  no worries at all! i'll be able to help out 100%! shoot me a dm, and i'll dive right in!  we don't want thi

In [12]:
print('\033[92m' + tweet) 

tweet2 = re.sub(r'^RT[\s]+', '', tweet)
 
tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

tweet2 = re.sub(r'#', '', tweet2)

emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  
    u"(\ud83c[\udf00-\uffff])|" 
    u"(\ud83d[\u0000-\uddff])|"  
    u"(\ud83d[\ude80-\udeff])|" 
    u"(\ud83c[\udde0-\uddff])"  
    "+", flags=re.UNICODE)

tweet2 = emoji_pattern.sub(r'', tweet2)
print('\033[94m' + tweet2) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
tweet2[:1000]

" hey, trevor! send us a dm, and we'll take a closer look!   thanks for getting that dm sent over - i've got it and am replying now. hit me up on this link &gt;&gt;please shoot me a dm   if you need any further assistance, you know where we are, 😃 😄  hey there.\n\nmy name is louise and i will be assisting you today.\n\nsorry to hear you having issues with the app,  would you like me to help with that payment arrangment?  let me take a look into this for you so we can get this resolved! t-force is here to help so send me a dm and we can get to the bottom of the issue! my name is louise and i am stepping in here for meghan.\n\ni think what meghan is suggesting is that you recevie your notifications directly through the twitter app instead of via text message,\n\ndoes that make sense?  let's take a closer look to see what's going on, laura. send us a dm!  hi iris. i know how important this is to get resolved as quickly as possible. you can always reach out in a dm here on twitter as well 

In [14]:
tweet = tweet2

In [15]:
sent_tokens=sent_tokenize(text=tweet)

In [16]:
sent_tokens[1]

"send us a dm, and we'll take a closer look!"

In [17]:
word_tokens = word_tokenize(tweet)

In [21]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punct_remove = dict((ord(punct), None) for punct in string.punctuation)

def perform_lemmatization(text):
    return lemmatization(nltk.word_tokenize(text.lower().translate(punct_remove)))

In [22]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    vectorizer = TfidfVectorizer(tokenizer=perform_lemmatization, stop_words='english')
    vectors = vectorizer.fit_transform(sent_tokens)
    vals = cosine_similarity(vectors[-1], vectors)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_diff = flat[-2]
    if(req_diff==0):
        robo_response=robo_response+"I have no answer for your question. I'm sorry! Anything else?"
        return robo_response
    else:
        robo_response = sent_tokens[idx]        
        return robo_response

In [28]:
dialogue=True

print("{name}: Welcome to {name} Customer Support Service. I will assist you with any problem related to {name}. To end this conversation, type bye!".format(name = organization))
while(dialogue==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='Thanks' or user_response=='Thank you' or user_response=='thanks' or user_response=='thank you'):
            print("{name}: You are welcome!".format(name = organization))
        else:
            print("{name}: ".format(name = organization), end="")
            print(response(user_response))
            sent_tokens.remove(user_response)
    else:
        dialogue=False
        print("{name}: Thank you for contacting us. I hope we could help you with your issue.".format(name = organization))

TMobileHelp: Welcome to TMobileHelp Customer Support Service. I will assist you with any problem related to TMobileHelp. To end this conversation, type bye!
Hi
TMobileHelp: hi
I encounter connectivity issues
TMobileHelp: t-force is here to help so send me a dm and we can get to the bottom of the issue!
also, my internet speed is very low
TMobileHelp: please dm me  i would love to take a look at that speed problem for you.
ok, i will send you a direct message
TMobileHelp: i definitely want to make sure we can get you... can you come over to a direct message please?
i will send you a dm!
TMobileHelp: send me a dm!
thank you
TMobileHelp: You are welcome!
bye
TMobileHelp: Thank you for contacting us. I hope we could help you with your issue.
